In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datasets/perovai_devices_day6_stability_ready.csv")
df.head()

,Device,FF_Forward,FF_Reverse,Jsc_mAcm2_Forward,Jsc_mAcm2_Reverse,PCE_percent_Forward,PCE_percent_Reverse,Rs_ohm_cm2_Forward,Rs_ohm_cm2_Reverse,Rsh_ohm_cm2_Forward,...,Voc_V_Forward,Voc_V_Reverse,Delta_PCE,Delta_Voc,Delta_Jsc,Delta_FF,Hysteresis_Index,High_Hysteresis,Low_Rs,Unstable
0,device001,0.650051,0.640161,20.177047,20.178596,14.491762,14.377952,8.245944,8.304344,5106.928620,...,1.104883,1.113055,-0.113810,0.008172,0.001549,-0.009890,-0.007916,0,1,0
1,device002,0.746704,0.758766,20.051937,20.154208,17.134683,17.777031,6.118376,7.594258,3341.636255,...,1.144383,1.162480,0.642347,0.018098,0.102270,0.012062,0.036134,0,1,0
2,device003,0.771115,0.767192,20.756446,20.797109,18.363962,18.387340,4.786209,5.584745,4110.186343,...,1.147345,1.152423,0.023378,0.005078,0.040663,-0.003923,0.001271,0,1,0
3,device004,0.748184,0.759476,20.071712,20.171704,17.191299,17.810883,6.110401,7.581881,3240.772843,...,1.144763,1.162596,0.619584,0.017833,0.099991,0.011292,0.034787,0,1,0
4,device005,0.748388,0.755708,20.152430,20.259536,17.333143,17.911775,6.452673,8.205573,3485.020804,...,1.149273,1.169918,0.578632,0.020645,0.107106,0.007320,0.032305,0,1,0


In [3]:
# Normalize key instability drivers
df["norm_Delta_PCE"] = (df["Delta_PCE"] - df["Delta_PCE"].mean()) / df["Delta_PCE"].std()
df["norm_Hysteresis"] = (df["Hysteresis_Index"] - df["Hysteresis_Index"].mean()) / df["Hysteresis_Index"].std()
df["norm_Rs"] = (df["Rs_ohm_cm2_Reverse"] - df["Rs_ohm_cm2_Reverse"].mean()) / df["Rs_ohm_cm2_Reverse"].std()

# Weighted risk score (based on Day‑6 coefficients qualitatively)
df["Risk_Score"] = (
    0.5 * df["norm_Delta_PCE"] +
    0.3 * df["norm_Hysteresis"] +
    0.2 * df["norm_Rs"]
)

df[["Device","Risk_Score"]].head()

,Device,Risk_Score
0,device001,-0.520716
1,device002,0.291022
2,device003,-0.373442
3,device004,0.266586
4,device005,0.222623


In [4]:
df = df.sort_values("Risk_Score", ascending=False)
df[["Device","Risk_Score"]].head(15)

,Device,Risk_Score
86,device087,2.794526
41,device042,2.204792
47,device048,1.155777
44,device045,0.958746
25,device026,0.814790
45,device046,0.809948
87,device088,0.777767
23,device024,0.759765
83,device084,0.706719
21,device022,0.705775


In [5]:
q1 = df["Risk_Score"].quantile(0.33)
q2 = df["Risk_Score"].quantile(0.66)

def risk_label(x):
    if x > q2:
        return "High Risk"
    elif x > q1:
        return "Medium Risk"
    else:
        return "Low Risk"

df["Risk_Category"] = df["Risk_Score"].apply(risk_label)
df["Risk_Category"].value_counts()

Risk_Category
High Risk      36
Medium Risk    35
Low Risk       35
Name: count, dtype: int64

In [6]:
def recommendation(row):
    if row["Risk_Category"] == "High Risk":
        if row["Hysteresis_Index"] > 0.05:
            return "Reduce ionic migration / improve interface passivation"
        elif row["Rs_ohm_cm2_Reverse"] > df["Rs_ohm_cm2_Reverse"].median():
            return "Improve charge transport / contacts"
        else:
            return "General stability optimization needed"
    elif row["Risk_Category"] == "Medium Risk":
        return "Monitor stability, optimize interfaces"
    else:
        return "Device appears stable"

df["Recommendation"] = df.apply(recommendation, axis=1)

In [7]:
df[[
    "Device",
    "Risk_Category",
    "Risk_Score",
    "Hysteresis_Index",
    "Rs_ohm_cm2_Reverse",
    "Delta_PCE",
    "Recommendation"
]].head(20)

,Device,Risk_Category,Risk_Score,Hysteresis_Index,Rs_ohm_cm2_Reverse,Delta_PCE,Recommendation
86,device087,High Risk,2.794526,1810.071312,8.914104e+07,0.008557,Reduce ionic migration / improve interface pas...
41,device042,High Risk,2.204792,0.211110,1.045415e+01,2.424966,Reduce ionic migration / improve interface pas...
47,device048,High Risk,1.155777,0.907055,1.458819e+09,0.000005,Reduce ionic migration / improve interface pas...
44,device045,High Risk,0.958746,0.135702,1.231424e+01,1.264251,Reduce ionic migration / improve interface pas...
25,device026,High Risk,0.814790,0.178794,1.461828e+01,1.130071,Reduce ionic migration / improve interface pas...
45,device046,High Risk,0.809948,0.100393,9.313728e+00,1.125685,Reduce ionic migration / improve interface pas...
87,device088,High Risk,0.777767,0.216026,5.879503e+01,1.095520,Reduce ionic migration / improve interface pas...
23,device024,High Risk,0.759765,0.175148,1.580237e+01,1.078815,Reduce ionic migration / improve interface pas...
83,device084,High Risk,0.706719,0.250471,3.545254e+01,1.029276,Reduce ionic migration / improve interface pas...
21,device022,High Risk,0.705775,0.173126,1.511822e+01,1.028520,Reduce ionic migration / improve interface pas...


In [8]:
df.to_csv("datasets/perovai_devices_day7_ranked_recommendations.csv", index=False)
print("Saved: datasets/perovai_devices_day7_ranked_recommendations.csv")

Saved: datasets/perovai_devices_day7_ranked_recommendations.csv
